1. In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/. At the end of the project, your group will give a presentation of your findings.

Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
The data that we want is contained in the "Total Raised and Spent" table.
Make a DataFrame showing, for each candidate:
the candidate's name
the candidate's party
state
district number
whether the candidate was an incumbent
whether the candidate won the race
the total amount raised by that candidate (as a numeric variable)
the total amount spent by the candidate (as a numeric variable)

In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [23]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'
response = requests.get(URL)
type(response)
response.status_code
#response.text
soup = BS(response.text)
#print(soup.prettify())


In [ ]:
df = pd.read

2. Once you have working code for Tennessee's 2nd District, expand on your code to capture all of Tennessee's districts.

3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

4. Using your scraped data, investigates different relationships between candidates and the amount of money they raised. Here are some suggestions to get you started, but feel free to pose you own questions or do additional exploration:
a. How often does the candidate who raised more money win a race?
b. How often does the candidate who spent more money win a race?
c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?
d. How often does the incumbent candidate win a race?
e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?